In [1]:
#这个模型是为了融合，B榜单模五折862在后一个模型
import toad
import pandas as pd
import numpy as np
import lightgbm as lgb
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
#import xgboost as xgb
import matplotlib.pyplot as plt
import gc

import warnings
warnings.simplefilter('ignore')

In [2]:
train = pd.read_excel("./data/train.xlsx",engine='openpyxl')
test = pd.read_excel('./data/test_A榜.xlsx',engine='openpyxl')
test_b = pd.read_excel('./data/test_B榜.xlsx',engine='openpyxl')
train.replace("?",np.nan,inplace=True)
test.replace("?",np.nan,inplace=True)
test_b.replace("?",np.nan,inplace=True)
train.to_csv('train_original.csv',index=0)
test.to_csv('test_a_original.csv',index=0)
test_b.to_csv('test_b_original.csv',index=0)

In [3]:
train_ = pd.read_csv('train_original.csv')
test_B = pd.read_csv('test_b_original.csv')
sub = pd.read_csv('data/sample_submit.txt')

In [4]:
train = train_.copy()
test = test_B.copy()

#修正
train['AGN_CUR_YEAR_AMT'] = train['AGN_CUR_YEAR_AMT'].apply(lambda x: np.log(x)-1.15)
train['AGN_CUR_YEAR_WAG_AMT'] = train['AGN_CUR_YEAR_WAG_AMT'].apply(lambda x: np.log(x)-1.15)

test['AGN_CUR_YEAR_AMT'] = test['AGN_CUR_YEAR_AMT'].apply(lambda x: np.log(x))
test['AGN_CUR_YEAR_WAG_AMT'] = test['AGN_CUR_YEAR_WAG_AMT'].apply(lambda x: np.log(x))
data = pd.concat([train,test]).reset_index(drop = True)

In [5]:
cols_drop=['ICO_CUR_MON_ACM_TRX_AMT', 'CUR_YEAR_MON_AGV_TRX_CNT',
           'MON_12_AGV_TRX_CNT', 'MON_12_ACM_LVE_ACT_CNT',
           'MON_12_AGV_LVE_ACT_CNT', 'CUR_MON_COR_DPS_MON_DAY_AVG_BAL',
           'CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL',
           'CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR', 'CUR_YEAR_MID_BUS_INC',
           'AI_STAR_SCO']
data = data.drop(cols_drop,axis = 1)

In [6]:
object_cols = ['MON_12_CUST_CNT_PTY_ID', 'WTHR_OPN_ONL_ICO', 'LGP_HLD_CARD_LVL','NB_CTC_HLD_IDV_AIO_CARD_SITU']
data[object_cols] = data[object_cols].astype('str')
data[object_cols] = data[object_cols].fillna('NULL')
for col in tqdm(object_cols):
    lbl = LabelEncoder()
    data[col] = lbl.fit_transform(data[col])

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
cols_corr =['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM',
           'NB_RCT_3_MON_LGN_TMS_AGV', 'COUNTER_CUR_YEAR_CNT_AMT',
           'PUB_TO_PRV_TRX_AMT_CUR_YEAR', 'MON_12_EXT_SAM_TRSF_IN_AMT',
           'MON_12_EXT_SAM_TRSF_OUT_AMT', 'MON_12_EXT_SAM_NM_TRSF_OUT_CNT',
           'MON_12_EXT_SAM_AMT', 'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT',
           'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT', 'MON_12_ACM_ENTR_ACT_CNT',
           'MON_12_AGV_ENTR_ACT_CNT', 'CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT',
           'MON_6_50_UP_ENTR_ACT_CNT', 'MON_6_50_UP_LVE_ACT_CNT',
           'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY',
           'MON_12_ACT_IN_50_UP_CNT_PTY_QTY',
           'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL',
           'LAST_12_MON_COR_DPS_DAY_AVG_BAL',
           'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV',
           'LAST_12_MON_MON_AVG_TRX_AMT_NAV', 'EMP_NBR', 'REG_CPT', 'REG_DT',
           'OPN_TM']
df = data.copy()
for col in cols_corr:
    dict_tuple = abs(df.corr()[col]).sort_values(ascending = False)
    colcorr = [key for key,vle in dict_tuple.items() if vle>0.4 and vle<0.8]
    if len(colcorr)>0:
        for i in colcorr:
            data[f'{col}_{i}_sub'] = data[col] - data[i]
            data[f'{col}_{i}_div'] = data[col] / (data[i]+1e-5)
    else:
        continue
        
data['isnull_mean'] = data.drop(['LABEL'],axis = 1).isnull().mean(axis = 1)   

In [8]:
# 删除特征重要性低的特征
cols_drops =  ['MON_12_EXT_SAM_AMT_MON_12_EXT_SAM_TRSF_IN_AMT_sub',
               'MON_12_EXT_SAM_TRSF_IN_AMT_MON_12_EXT_SAM_AMT_sub',
               'MON_12_ACT_IN_50_UP_CNT_PTY_QTY_MON_12_ACT_OUT_50_UP_CNT_PTY_QTY_sub',
               'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_MON_12_EXT_SAM_TRSF_OUT_AMT_sub',
               'NB_CTC_HLD_IDV_AIO_CARD_SITU',
               'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY_MON_12_EXT_SAM_TRSF_OUT_AMT_sub',
               'COUNTER_CUR_YEAR_CNT_AMT_CUR_YEAR_COUNTER_ENCASH_CNT_div',
               'MON_12_EXT_SAM_TRSF_OUT_AMT',
               'MON_12_EXT_SAM_AMT_MON_12_EXT_SAM_TRSF_OUT_AMT_sub',
               'MON_12_EXT_SAM_TRSF_OUT_AMT_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_sub',
               'WTHR_OPN_ONL_ICO',
               'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY_MON_12_EXT_SAM_TRSF_IN_AMT_sub',
               'MON_12_EXT_SAM_TRSF_IN_AMT_AGN_CUR_YEAR_AMT_sub',
               'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_AGN_CUR_YEAR_AMT_sub',
               'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_MON_12_EXT_SAM_AMT_sub',
               'MON_12_EXT_SAM_TRSF_OUT_AMT_AGN_CUR_YEAR_WAG_AMT_div',
               'MON_12_EXT_SAM_TRSF_OUT_AMT_MON_12_EXT_SAM_AMT_sub',
               'MON_12_EXT_SAM_AMT_AGN_CUR_YEAR_AMT_sub',
               'MON_12_EXT_SAM_AMT_AGN_CUR_YEAR_WAG_AMT_div',
               'LAST_12_MON_COR_DPS_DAY_AVG_BAL_MON_12_EXT_SAM_TRSF_IN_AMT_sub']

In [9]:
train = data[data['LABEL'].notnull()]
test = data[~data['LABEL'].notnull()].reset_index(drop = True)

cols_psi_drop = ['ICO_CUR_MON_ACM_TRX_TM', 'OPN_TM', 'REG_DT',
       'MON_12_EXT_SAM_NM_TRSF_OUT_CNT_ICO_CUR_MON_ACM_TRX_TM_div',
       'ICO_CUR_MON_ACM_TRX_TM_MON_12_EXT_SAM_NM_TRSF_OUT_CNT_div']
feas = [i  for i in train.columns.tolist() if i not in ['CUST_UID','LABEL',]+cols_psi_drop+cols_drops]
x_train = train[feas]
y_train = train['LABEL']
x_test = test[feas]

In [10]:
feat_lst = list(data[feas])  
psi_df = toad.metrics.PSI(x_train[feat_lst], x_test[feat_lst]).sort_values(0)  


psi_df[psi_df>0.3].index

Index(['isnull_mean'], dtype='object')

In [11]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2022
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seeds)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []
    test_pre = []
    Feass = pd.DataFrame()

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        
        
        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)#, weight=df_train_copy['is_test_prob'][train_index]
            valid_matrix = clf.Dataset(val_x, label=val_y)
            
            
            fea = pd.DataFrame()

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'tree_learner':'serial',
                'metric': 'AUC',
                'min_child_weight': 4,
                'num_leaves': 2 ** 4,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.0045,#0.0033
#                 'reg_alpha':0.001,
#                 'reg_lambda':10,
#                 'device':'gpu',
#                 'gpu_platform_id':0, 
#                 'gpu_device_id':0,
                'seed': seed,
                'nthread': 28,
                'n_jobs':-1,
                'silent': True,
                'verbose': -1,
            }

            model = clf.train(params, train_matrix, 1000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature =[] ,verbose_eval=200,early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            test_pre.append(test_pred)
            fea['feas'] = train_x.columns.tolist()
            fea['sorce'] = model.feature_importance("gain")
            Feass = pd.concat([Feass,fea],axis = 0)
            print(list(sorted(zip(train_x.columns.tolist(), model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
        
        
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix  = clf.DMatrix(test_x)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'min_child_weight': 1.5,
                      'max_depth': 4,
                      'subsample': 0.8,
                      'colsample_bytree': 0.8,
                      'colsample_bylevel': 0.8,
                      'eta': 0.003,
                      'tree_method': 'gpu_hist',
                      'seed': 2022,
                      'nthread': 36,
                      "silent": True,
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=3000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
            val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(test_matrix , ntree_limit=model.best_ntree_limit)
            test_pre.append(test_pred)

        
            
        train[valid_index] = val_pred
        test = test_pred
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        
        print(cv_scores)
    test = sum(test_pre) / folds
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    if clf_name=='lgb':
        return train, test, Feass
    else:
        return train, test
def lgb_model(x_train, y_train, x_test):
    lgb_train, lgb_test ,Feass= cv_model(lgb, x_train, y_train, x_test, "lgb")
    return lgb_train, lgb_test, Feass

def xgb_model(x_train, y_train, x_test):
    xgb_train, xgb_test = cv_model(xgb, x_train, y_train, x_test, "xgb")
    return xgb_train, xgb_test

In [12]:
seeds = 2022
lgb_train, lgb_test_2022, Feass = lgb_model(x_train, y_train, x_test)
seeds = 4466
lgb_train, lgb_test_4466, Feass = lgb_model(x_train, y_train, x_test)
seeds = 1234
lgb_train, lgb_test_1234, Feass = lgb_model(x_train, y_train, x_test)

************************************ 1 ************************************
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=28 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.859577	valid_1's auc: 0.850754
[400]	training's auc: 0.865111	valid_1's auc: 0.855381
[600]	training's auc: 0.870265	valid_1's auc: 0.859092
[800]	training's auc: 0.875263	valid_1's auc: 0.862164
[1000]	training's auc: 0.879586	valid_1's auc: 0.864054
Did not meet early stopping. Best iteration is:
[1000]	training's auc: 0.879586	valid_1's auc: 0.864054
[('LAST_12_MON_COR_DPS_DAY_AVG_BAL', 706611.0887041092), ('CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub', 58616.91875743866), ('ICO_CUR_MON_ACM_TRX_TM_MON_12_EXT_SAM_NM_TRSF_OUT_CNT_sub', 28636.973837852478), ('LAST_12_MON_COR_DPS_DAY_AVG_BAL_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_sub', 26404.2099123

Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.860224	valid_1's auc: 0.850555
[400]	training's auc: 0.865671	valid_1's auc: 0.854216
[600]	training's auc: 0.870827	valid_1's auc: 0.857432
[800]	training's auc: 0.875748	valid_1's auc: 0.859976
[1000]	training's auc: 0.880016	valid_1's auc: 0.86187
Did not meet early stopping. Best iteration is:
[1000]	training's auc: 0.880016	valid_1's auc: 0.86187
[('LAST_12_MON_COR_DPS_DAY_AVG_BAL', 707205.8578948975), ('CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub', 81289.18545341492), ('ICO_CUR_MON_ACM_TRX_TM_MON_12_EXT_SAM_NM_TRSF_OUT_CNT_sub', 34440.30711746216), ('LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL_LAST_12_MON_MON_AVG_TRX_AMT_NAV_div', 20403.77103614807), ('MON_6_50_UP_ENTR_ACT_CNT_MON_12_ACT_IN_50_UP_CNT_PTY_QTY_div', 18986.15873527527), ('MON_6_50_UP_ENTR_ACT_CNT', 17656.434953689575), ('LAST_12_MON_MON_AVG_TRX_AMT_NAV_LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL_div', 14986.241479

Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.861295	valid_1's auc: 0.846358
[400]	training's auc: 0.866631	valid_1's auc: 0.850213
[600]	training's auc: 0.871716	valid_1's auc: 0.853262
[800]	training's auc: 0.876537	valid_1's auc: 0.855971
[1000]	training's auc: 0.880886	valid_1's auc: 0.857959
Did not meet early stopping. Best iteration is:
[1000]	training's auc: 0.880886	valid_1's auc: 0.857959
[('LAST_12_MON_COR_DPS_DAY_AVG_BAL', 716424.0291185379), ('CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub', 57951.36565208435), ('ICO_CUR_MON_ACM_TRX_TM_MON_12_EXT_SAM_NM_TRSF_OUT_CNT_sub', 30553.01762008667), ('LAST_12_MON_COR_DPS_DAY_AVG_BAL_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_sub', 27529.007900238037), ('MON_6_50_UP_ENTR_ACT_CNT_MON_12_ACT_IN_50_UP_CNT_PTY_QTY_div', 18654.062598228455), ('LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL_LAST_12_MON_MON_AVG_TRX_AMT_NAV_div', 18569.04971599579), ('LAST_12_MON_COR_DPS_DAY_AVG_BAL_CUR_MON_EX

Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.859412	valid_1's auc: 0.856424
[400]	training's auc: 0.864597	valid_1's auc: 0.860074
[600]	training's auc: 0.869962	valid_1's auc: 0.86333
[800]	training's auc: 0.87494	valid_1's auc: 0.866
[1000]	training's auc: 0.87917	valid_1's auc: 0.867826
Did not meet early stopping. Best iteration is:
[1000]	training's auc: 0.87917	valid_1's auc: 0.867826
[('LAST_12_MON_COR_DPS_DAY_AVG_BAL', 703885.4458370209), ('LAST_12_MON_COR_DPS_DAY_AVG_BAL_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_sub', 47849.719135284424), ('CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub', 41910.216386795044), ('ICO_CUR_MON_ACM_TRX_TM_MON_12_EXT_SAM_NM_TRSF_OUT_CNT_sub', 33465.09408950806), ('LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL_LAST_12_MON_MON_AVG_TRX_AMT_NAV_div', 19232.29305744171), ('MON_6_50_UP_ENTR_ACT_CNT_MON_12_ACT_IN_50_UP_CNT_PTY_QTY_div', 17231.015283584595), ('MON_6_50_UP_ENTR_ACT_CNT', 16768.975708961487), 

In [15]:
lgb_test_2022,lgb_test_4466,lgb_test_1234,(lgb_test_2022+lgb_test_4466+lgb_test_1234)/3

(array([0.10975707, 0.02350036, 0.13972814, ..., 0.1730631 , 0.0622355 ,
        0.48538906]),
 array([0.11205303, 0.02301328, 0.13689846, ..., 0.17189233, 0.06180734,
        0.49518273]),
 array([0.10960416, 0.02296019, 0.13506645, ..., 0.1714313 , 0.0619402 ,
        0.4825715 ]),
 array([0.11047142, 0.02315794, 0.13723101, ..., 0.17212891, 0.06199435,
        0.48771443]))

In [16]:
sub = test[['CUST_UID','LABEL']]
sub['LABEL']= (lgb_test_2022+lgb_test_4466+lgb_test_1234)/3
sub.to_csv('lgb_B_3seed.txt', header=None,index=False,encoding = 'utf-8',sep = '\t')

In [20]:
sub

,CUST_UID,LABEL
0,f5dcfe9b9e8d496d893cfcba8fcac1d4,0.110471
1,68883d45646d44d8b851476058e4500d,0.023158
2,2b93dfa58f10463a937cd42cc42e54f7,0.137231
3,94fc13e16dac4642ab7544d8889e8579,0.019002
4,6be5af4911964a31a0b5cf115799ace6,0.202422
...,...,...
11995,01d799cc3246416387f2ff9a1468ede1,0.090817
11996,55dba67ea8eb464a810c4fea9331e443,0.092831
11997,527e025840b14373890844a1138197ab,0.172129
11998,89a4e87c9a764c6c9eafda790c76a4e6,0.061994


In [26]:
#第二个模型
#单模五折862

In [11]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score,f1_score,recall_score
pd.set_option('display.max_rows', None)#显示全部行
pd.set_option('display.max_columns', None)#显示全部列
import warnings
import pickle

import gc
from tqdm import tqdm
import os
import toad
warnings.filterwarnings('ignore')

In [12]:
train = pd.read_csv('train_original.csv')
#test = pd.read_csv('test_a_original.csv')
test = pd.read_csv('test_b_original.csv')
test['LABEL']=-1

In [13]:
train['AGN_CUR_YEAR_AMT'] = train['AGN_CUR_YEAR_AMT'].apply(lambda x: np.log(x)-1.15)
train['AGN_CUR_YEAR_WAG_AMT'] = train['AGN_CUR_YEAR_WAG_AMT'].apply(lambda x: np.log(x)-1.15)

test['AGN_CUR_YEAR_AMT'] = test['AGN_CUR_YEAR_AMT'].apply(lambda x: np.log(x))
test['AGN_CUR_YEAR_WAG_AMT'] = test['AGN_CUR_YEAR_WAG_AMT'].apply(lambda x: np.log(x))

In [14]:
data = pd.concat([train,test]).reset_index(drop=True)
data.replace("?",np.nan,inplace=True)
print(train['LABEL'].value_counts())

0    30000
1    10000
Name: LABEL, dtype: int64


In [15]:
py_col = ['CUR_YEAR_MON_AGV_TRX_CNT','CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL','CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR',
          'CUR_YEAR_MID_BUS_INC','AI_STAR_SCO','CUR_MON_COR_DPS_MON_DAY_AVG_BAL','ICO_CUR_MON_ACM_TRX_AMT']

gb_col = ['MON_12_AGV_TRX_CNT','MON_12_ACM_LVE_ACT_CNT','MON_12_AGV_LVE_ACT_CNT']
data.drop(py_col,axis=1,inplace=True)
data.drop(gb_col,axis=1,inplace=True)

In [16]:
data['PUB_TO_PRV_TRX_AMT_CUR_month'] = data['PUB_TO_PRV_TRX_AMT_CUR_YEAR']/12

In [17]:
le_col = []
for col in data.columns:
    if data[col].dtype=='object' and col!="CUST_UID":
        print(col)
        #print(data[col].nunique)
        le=LabelEncoder()
        data[col]=data[col].astype("str")
        data[col]=le.fit_transform(data[col])
        le_col.append(col)

MON_12_CUST_CNT_PTY_ID
WTHR_OPN_ONL_ICO
LGP_HLD_CARD_LVL
NB_CTC_HLD_IDV_AIO_CARD_SITU


In [18]:
#data.replace(np.nan,-9e+13,inplace=True)
df_train = data[data['LABEL']!=-1].reset_index(drop=True)
df_test  = data[data['LABEL']==-1].reset_index(drop=True)

print(df_train.shape,df_test.shape)

(40000, 42) (12000, 42)


In [19]:
le_col+=['LABEL','CUST_UID','MON_12_CUST_CNT_PTY_ID']
le_col

['MON_12_CUST_CNT_PTY_ID',
 'WTHR_OPN_ONL_ICO',
 'LGP_HLD_CARD_LVL',
 'NB_CTC_HLD_IDV_AIO_CARD_SITU',
 'LABEL',
 'CUST_UID',
 'MON_12_CUST_CNT_PTY_ID']

In [20]:
column = [i for i in df_train.columns if i not in le_col and df_train[i].std() != 0]
len(column)

36

In [21]:
df = data.copy()
for col in column:
    dic = abs(df.corr()[col]).sort_values(ascending = False)
    l = [k for k,v in dic.items() if v>0.35 and v<0.85]
    if len(l)>0:
        for i in l:
            data[f'{col}_{i}_sub'] = data[col] - data[i]
            data[f'{col}_{i}_diff'] = data[col] / data[i]

    p = [k for k,v in dic.items() if v>0.7 and v<0.96]
    if len(l)>0:
        for i in p:
            data[f'{col}_{i}_add'] = data[col] + data[i]
            data[f'{i}_rate'] = data[i] / data[f'{col}_{i}_add']
            data[f'{col}_rate'] = data[col] / data[f'{col}_{i}_add']
    print(col)

AGN_CNT_RCT_12_MON
ICO_CUR_MON_ACM_TRX_TM
NB_RCT_3_MON_LGN_TMS_AGV
AGN_CUR_YEAR_AMT
AGN_CUR_YEAR_WAG_AMT
AGN_AGR_LATEST_AGN_AMT
COUNTER_CUR_YEAR_CNT_AMT
PUB_TO_PRV_TRX_AMT_CUR_YEAR
MON_12_EXT_SAM_TRSF_IN_AMT
MON_12_EXT_SAM_TRSF_OUT_AMT
MON_12_EXT_SAM_NM_TRSF_OUT_CNT
MON_12_EXT_SAM_AMT
CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT
CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT
MON_12_TRX_AMT_MAX_AMT_PCTT
MON_12_ACM_ENTR_ACT_CNT
MON_12_AGV_ENTR_ACT_CNT
CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT
MON_6_50_UP_ENTR_ACT_CNT
MON_6_50_UP_LVE_ACT_CNT
CUR_YEAR_COUNTER_ENCASH_CNT
MON_12_ACT_OUT_50_UP_CNT_PTY_QTY
MON_12_ACT_IN_50_UP_CNT_PTY_QTY
LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL
LAST_12_MON_COR_DPS_DAY_AVG_BAL
LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV
LAST_12_MON_MON_AVG_TRX_AMT_NAV
COR_KEY_PROD_HLD_NBR
EMP_NBR
REG_CPT
SHH_BCK
HLD_DMS_CCY_ACT_NBR
REG_DT
OPN_TM
HLD_FGN_CCY_ACT_NBR
PUB_TO_PRV_TRX_AMT_CUR_month


In [22]:
#IN/OUT比例
data['fea_1'] = data['MON_12_ACT_IN_50_UP_CNT_PTY_QTY']/data['MON_12_ACT_OUT_50_UP_CNT_PTY_QTY']
data['fea_2'] = data['MON_12_EXT_SAM_TRSF_IN_AMT']/data['MON_12_EXT_SAM_TRSF_OUT_AMT']
data['fea_3'] = data['CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT']/data['CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT']

#ACT/LVE比例
#data['fea_4']=data['MON_12_ACM_ENTR_ACT_CNT']/data['MON_12_ACM_LVE_ACT_CNT']
#data['fea_5']=data['MON_12_AGV_ENTR_ACT_CNT']/data['MON_12_AGV_LVE_ACT_CNT']
data['fea_6']=data['MON_6_50_UP_ENTR_ACT_CNT']/data['MON_6_50_UP_LVE_ACT_CNT']

#data['fea_7']=(data['CUR_MON_COR_DPS_MON_DAY_AVG_BAL']*12)/data['LAST_12_MON_COR_DPS_DAY_AVG_BAL']
data['fea_8']=data['MON_12_EXT_SAM_TRSF_IN_AMT']/data['MON_12_EXT_SAM_TRSF_OUT_AMT']
data['fea_9']=data['CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT']/data['CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT']
data['fea_10']=data['MON_6_50_UP_ENTR_ACT_CNT']/data['MON_6_50_UP_LVE_ACT_CNT']
data['fea_11']=data['MON_12_ACT_IN_50_UP_CNT_PTY_QTY']/data['MON_12_ACT_OUT_50_UP_CNT_PTY_QTY']
#data['fea_12']=data['MON_12_ACM_ENTR_ACT_CNT']/data['MON_12_ACM_LVE_ACT_CNT']
#data['fea_13']=data['MON_12_AGV_ENTR_ACT_CNT']/data['MON_12_AGV_LVE_ACT_CNT']
#data['fea_14']=data['LAST_12_MON_COR_DPS_DAY_AVG_BAL']/(data['CUR_MON_COR_DPS_MON_DAY_AVG_BAL']*12)
data.head()

,CUST_UID,LABEL,AGN_CNT_RCT_12_MON,ICO_CUR_MON_ACM_TRX_TM,NB_RCT_3_MON_LGN_TMS_AGV,AGN_CUR_YEAR_AMT,AGN_CUR_YEAR_WAG_AMT,AGN_AGR_LATEST_AGN_AMT,COUNTER_CUR_YEAR_CNT_AMT,PUB_TO_PRV_TRX_AMT_CUR_YEAR,MON_12_EXT_SAM_TRSF_IN_AMT,MON_12_EXT_SAM_TRSF_OUT_AMT,MON_12_EXT_SAM_NM_TRSF_OUT_CNT,MON_12_EXT_SAM_AMT,CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT,CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT,MON_12_CUST_CNT_PTY_ID,MON_12_TRX_AMT_MAX_AMT_PCTT,MON_12_ACM_ENTR_ACT_CNT,MON_12_AGV_ENTR_ACT_CNT,CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT,MON_6_50_UP_ENTR_ACT_CNT,MON_6_50_UP_LVE_ACT_CNT,CUR_YEAR_COUNTER_ENCASH_CNT,MON_12_ACT_OUT_50_UP_CNT_PTY_QTY,MON_12_ACT_IN_50_UP_CNT_PTY_QTY,LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL,LAST_12_MON_COR_DPS_DAY_AVG_BAL,LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV,LAST_12_MON_MON_AVG_TRX_AMT_NAV,COR_KEY_PROD_HLD_NBR,WTHR_OPN_ONL_ICO,EMP_NBR,REG_CPT,SHH_BCK,HLD_DMS_CCY_ACT_NBR,REG_DT,LGP_HLD_CARD_LVL,OPN_TM,NB_CTC_HLD_IDV_AIO_CARD_SITU,HLD_FGN_CCY_ACT_NBR,PUB_TO_PRV_TRX_AMT_CUR_month,ICO_CUR_MON_ACM_TRX_TM_MON_12_EXT_SAM_NM_TRSF_OUT_CNT_sub,ICO_CUR_MON_ACM_TRX_TM_MON_12_EXT_SAM_NM_TRSF_OUT_CNT_diff,ICO_CUR_MON_ACM_TRX_TM_CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT_sub,ICO_CUR_MON_ACM_TRX_TM_CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT_diff,ICO_CUR_MON_ACM_TRX_TM_MON_12_EXT_SAM_NM_TRSF_OUT_CNT_add,MON_12_EXT_SAM_NM_TRSF_OUT_CNT_rate,ICO_CUR_MON_ACM_TRX_TM_rate,AGN_CUR_YEAR_WAG_AMT_MON_12_CUST_CNT_PTY_ID_sub,AGN_CUR_YEAR_WAG_AMT_MON_12_CUST_CNT_PTY_ID_diff,AGN_CUR_YEAR_WAG_AMT_AGN_CUR_YEAR_AMT_add,AGN_CUR_YEAR_AMT_rate,AGN_CUR_YEAR_WAG_AMT_rate,AGN_AGR_LATEST_AGN_AMT_MON_12_EXT_SAM_AMT_sub,AGN_AGR_LATEST_AGN_AMT_MON_12_EXT_SAM_AMT_diff,AGN_AGR_LATEST_AGN_AMT_MON_12_EXT_SAM_TRSF_IN_AMT_sub,AGN_AGR_LATEST_AGN_AMT_MON_12_EXT_SAM_TRSF_IN_AMT_diff,COUNTER_CUR_YEAR_CNT_AMT_CUR_YEAR_COUNTER_ENCASH_CNT_sub,COUNTER_CUR_YEAR_CNT_AMT_CUR_YEAR_COUNTER_ENCASH_CNT_diff,MON_12_EXT_SAM_TRSF_IN_AMT_MON_12_EXT_SAM_AMT_sub,MON_12_EXT_SAM_TRSF_IN_AMT_MON_12_EXT_SAM_AMT_diff,MON_12_EXT_SAM_TRSF_IN_AMT_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_sub,MON_12_EXT_SAM_TRSF_IN_AMT_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_diff,MON_12_EXT_SAM_TRSF_IN_AMT_LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL_sub,MON_12_EXT_SAM_TRSF_IN_AMT_LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL_diff,MON_12_EXT_SAM_TRSF_IN_AMT_MON_12_ACT_OUT_50_UP_CNT_PTY_QTY_sub,MON_12_EXT_SAM_TRSF_IN_AMT_MON_12_ACT_OUT_50_UP_CNT_PTY_QTY_diff,MON_12_EXT_SAM_TRSF_IN_AMT_AGN_AGR_LATEST_AGN_AMT_sub,MON_12_EXT_SAM_TRSF_IN_AMT_AGN_AGR_LATEST_AGN_AMT_diff,MON_12_EXT_SAM_TRSF_IN_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub,MON_12_EXT_SAM_TRSF_IN_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_diff,MON_12_EXT_SAM_TRSF_IN_AMT_LAST_12_MON_MON_AVG_TRX_AMT_NAV_sub,MON_12_EXT_SAM_TRSF_IN_AMT_LAST_12_MON_MON_AVG_TRX_AMT_NAV_diff,MON_12_EXT_SAM_TRSF_IN_AMT_MON_12_EXT_SAM_TRSF_OUT_AMT_add,MON_12_EXT_SAM_TRSF_OUT_AMT_rate,MON_12_EXT_SAM_TRSF_IN_AMT_rate,MON_12_EXT_SAM_TRSF_IN_AMT_CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_add,CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_rate,MON_12_EXT_SAM_TRSF_IN_AMT_MON_12_EXT_SAM_AMT_add,MON_12_EXT_SAM_AMT_rate,MON_12_EXT_SAM_TRSF_IN_AMT_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_add,CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_rate,MON_12_EXT_SAM_TRSF_OUT_AMT_CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_sub,MON_12_EXT_SAM_TRSF_OUT_AMT_CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_diff,MON_12_EXT_SAM_TRSF_OUT_AMT_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_sub,MON_12_EXT_SAM_TRSF_OUT_AMT_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_diff,MON_12_EXT_SAM_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL_sub,MON_12_EXT_SAM_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL_diff,MON_12_EXT_SAM_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub,MON_12_EXT_SAM_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_diff,MON_12_EXT_SAM_TRSF_OUT_AMT_MON_12_EXT_SAM_AMT_sub,MON_12_EXT_SAM_TRSF_OUT_AMT_MON_12_EXT_SAM_AMT_diff,MON_12_EXT_SAM_TRSF_OUT_AMT_MON_12_ACT_OUT_50_UP_CNT_PTY_QTY_sub,MON_12_EXT_SAM_TRSF_OUT_AMT_MON_12_ACT_OUT_50_UP_CNT_PTY_QTY_diff,MON_12_EXT_SAM_TRSF_OUT_AMT_LAST_12_MON_MON_AVG_TRX_AMT_NAV_sub,MON_12_EXT_SAM_TRSF_OUT_AMT_LAST_12_MON_MON_AVG_TRX_AMT

In [23]:
def get_features(df):
    high_importance_diff = [['REG_DT','OPN_TM'],
                            #['PUB_TO_PRV_TRX_AMT_CUR_MON','ICO_CUR_MON_ACM_TRX_AMT'],
    #['CUR_YEAR_MID_BUS_INC','NB_RCT_3_MON_LGN_TMS_AGV'],
    #['PUB_TO_PRV_TRX_AMT_CUR_MON','CUR_YEAR_MON_AGV_TRX_CNT'], 
    #['CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR','LAST_12_MON_COR_DPS_DAY_AVG_BAL'],
    #['CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR','CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL'], #日均增长率
    ]
    hand_IO_fea = [['MON_12_ACT_IN_50_UP_CNT_PTY_QTY','MON_12_ACT_OUT_50_UP_CNT_PTY_QTY'],['MON_6_50_UP_ENTR_ACT_CNT','MON_12_EXT_SAM_TRSF_OUT_AMT']
    ,['CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT','CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT'],['MON_6_50_UP_ENTR_ACT_CNT','MON_6_50_UP_LVE_ACT_CNT'],
    ['MON_12_ACM_ENTR_ACT_CNT','MON_12_ACM_LVE_ACT_CNT'],['MON_12_AGV_ENTR_ACT_CNT','MON_12_AGV_LVE_ACT_CNT']]

    hand_other_fea = [
    #['ICO_CUR_MON_ACM_TRX_AMT','ICO_CUR_MON_ACM_TRX_TM'],
    ['MON_12_EXT_SAM_AMT','MON_12_EXT_SAM_NM_TRSF_OUT_CNT'],
    #['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL','CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR'],
    ['LAST_12_MON_COR_DPS_DAY_AVG_BAL','LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL'],
    ['LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV','LAST_12_MON_MON_AVG_TRX_AMT_NAV'],['AGN_CNT_RCT_12_MON','AGN_AGR_LATEST_AGN_AMT']]
    for fea in high_importance_diff:
        df[f'{fea[0]}_{fea[1]}_diff'] = df[fea[0]] - df[fea[1]]
        
    for fea in hand_IO_fea and hand_other_fea:
        df[f'{fea[0]}_{fea[1]}_sub'] = df[fea[0]] / df[fea[1]]
        df[f'{fea[0]}_{fea[1]}_diff'] = df[fea[0]] - df[fea[1]]

    return df

data = get_features(data)

In [24]:
#data.replace(np.nan,-9e+13,inplace=True)
df_train = data[data['LABEL']!=-1].reset_index(drop=True)
df_test  = data[data['LABEL']==-1].reset_index(drop=True)

print(df_train.shape,df_test.shape)

(40000, 297) (12000, 297)


In [25]:
feats=[i for i in df_train.columns if i not in ['LABEL','CUST_UID']]

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
#from notebook import tqdm
X_train = df_train[feats]
X_test = df_test[feats]

features = X_train.columns.tolist()
X_train['risk_label'] = 1
X_test['risk_label'] = 0
train_test = pd.concat([X_train, X_test], axis =0,ignore_index=True)
train1, test1 = train_test_split(train_test, test_size=0.33, random_state=42, shuffle=True)
train_y = train1['risk_label'].values
test_y = test1['risk_label'].values
del train1['risk_label'], test1['risk_label']

if 'risk_label' in features:
    features.remove('risk_label')

adversarial_result = pd.DataFrame(index=train1.columns,columns=['roc'])
for i in tqdm(features):
    clf = lgb.LGBMClassifier(
        random_state=47,
        max_depth=2,
        metric='auc',
        n_estimators=1000,
        importance_type='gain'
    )
    clf.fit(
        np.array(train1[i]).reshape(-1,1),
        train_y,
        eval_set=[(np.array(test1[i]).reshape(-1,1),test_y)],
        early_stopping_rounds=200,
        verbose=0)
    temp_pred = clf.predict_proba(np.array(test1[i]).reshape(-1,1))[:,1]
    roc1 = roc_auc_score(test_y,temp_pred)
    adversarial_result.loc[i,'roc'] = roc1
    
a = adversarial_result.sort_values('roc', ascending=False)
a

100%|█████████████████████████████████████████| 295/295 [02:45<00:00,  1.78it/s]


,roc
PUB_TO_PRV_TRX_AMT_CUR_YEAR,0.628119
PUB_TO_PRV_TRX_AMT_CUR_month,0.628119
CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_diff,0.626835
LAST_12_MON_COR_DPS_DAY_AVG_BAL_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_diff,0.626487
LAST_12_MON_COR_DPS_DAY_AVG_BAL_CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_sub,0.626217
CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_diff,0.626128
LAST_12_MON_COR_DPS_DAY_AVG_BAL_CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_diff,0.625862
LAST_12_MON_COR_DPS_DAY_AVG_BAL_CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_sub,0.624743
CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub,0.62459
CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub,0.624408


In [27]:
#col = list(a.tail(210).index)
col = list(a.tail(210).index)
len(col)

210

In [28]:
col+=['LABEL','CUST_UID']
data = data[col]

In [29]:
#data.replace(np.nan,-9e+13,inplace=True)
df_train = data[data['LABEL']!=-1].reset_index(drop=True)


df_test  = data[data['LABEL']==-1].reset_index(drop=True)

print(df_train.shape,df_test.shape)

(40000, 212) (12000, 212)


In [30]:
fc = []
drop_fea = []
feats=[i for i in df_train.columns if i not in ['LABEL','CUST_UID','MON_12_CUST_CNT_PTY_ID']]
for col in feats:
    s = df_train[col].std()
    if s in fc or s==0:
        drop_fea.append(col)
    else:
        fc.append(s)

In [31]:
#删除毒特
#data['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL_rate'] = data['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL']/train['CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR']
#data['LAST_12_MON_COR_DPS_DAY_AVG_BAL_rate'] = data['LAST_12_MON_COR_DPS_DAY_AVG_BAL']/train['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL']
#data['LAST_12_MON_COR_DPS_DAY_AVG_BAL_diff'] = data['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL'] - train['LAST_12_MON_COR_DPS_DAY_AVG_BAL']
data['LAST_12_MON_MON_AVG_TRX_AMT_NAV_rate'] = data['LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV']/train['LAST_12_MON_MON_AVG_TRX_AMT_NAV']
data['AGN_CNT_RCT_12_MON_rate'] = data['AGN_CNT_RCT_12_MON']/data['AGN_AGR_LATEST_AGN_AMT']

col = [
      'LAST_12_MON_MON_AVG_TRX_AMT_NAV_rate','AGN_CNT_RCT_12_MON_rate']
s = []
for c in col:
    s.append(data[c].std())

In [32]:
df_train = data[data['LABEL']!=-1].reset_index(drop=True)
df_test  = data[data['LABEL']==-1].reset_index(drop=True)

In [33]:
feats=[i for i in df_train.columns if i not in ['LABEL','CUST_UID','MON_12_CUST_CNT_PTY_ID']+drop_fea]
len(feats)

160

In [34]:
feat=[i for i in feats if i not in drop_fea]
print(len(feat))

160


In [35]:
yd_col = []

In [36]:
for col in feats:
    if data[col].std() in s:
        yd_col.append(col)
yd_col

['LAST_12_MON_MON_AVG_TRX_AMT_NAV_rate',
 'AGN_CNT_RCT_12_MON_AGN_AGR_LATEST_AGN_AMT_sub',
 'AGN_CNT_RCT_12_MON_rate']

In [37]:
df_train = data[data['LABEL']!=-1].reset_index(drop=True)
df_test  = data[data['LABEL']==-1].reset_index(drop=True)

In [38]:
from catboost import CatBoostClassifier

In [39]:
import toad
feat_lst = [i for i in df_train.columns if i not in ['LABEL','CUST_UID','MON_12_CUST_CNT_PTY_ID']+drop_fea+yd_col]
psi_df = toad.metrics.PSI(df_train[feat_lst],df_test[feat_lst]).sort_values(0)  
psi_df

AGN_CUR_YEAR_WAG_AMT_AGN_CUR_YEAR_AMT_add                                        0.000331
AGN_CUR_YEAR_WAG_AMT_MON_12_CUST_CNT_PTY_ID_sub                                  0.000331
AGN_CUR_YEAR_WAG_AMT                                                             0.000331
AGN_CUR_YEAR_WAG_AMT_rate                                                        0.000331
AGN_CUR_YEAR_AMT_rate                                                            0.000331
MON_12_EXT_SAM_TRSF_IN_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub                   0.000440
MON_12_EXT_SAM_TRSF_IN_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_diff                  0.000440
LAST_12_MON_COR_DPS_DAY_AVG_BAL_MON_12_EXT_SAM_TRSF_IN_AMT_diff                  0.000440
LAST_12_MON_COR_DPS_DAY_AVG_BAL_MON_12_EXT_SAM_TRSF_OUT_AMT_diff                 0.000478
MON_12_EXT_SAM_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_diff                 0.000478
MON_12_EXT_SAM_TRSF_OUT_AMT_LAST_12_MON_COR_DPS_DAY_AVG_BAL_sub                  0.000478
AGN_CUR_YE

In [ ]:
feats=[i for i in df_train.columns if i not in ['LABEL','CUST_UID','MON_12_CUST_CNT_PTY_ID']+drop_fea+yd_col]
from sklearn.model_selection import KFold,StratifiedKFold
from lightgbm import LGBMClassifier
import lightgbm as lgb
print(len(feats))

train_label=df_train['LABEL']


def model_train(df_train,df_test,train_label,model_name,feat,seed,label_name,n_fold=10):

    skf=StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
    oof=np.zeros(len(df_train))
    prediction=np.zeros(len(df_test))
    oof = np.zeros(len(df_train))
    train_label=df_train[label_name]
    i=0
    importance = 0
    pred_y = 0
    def eval(y_hat,data):
        y_true = data.get_label()
        return 'f1_score', f1_score(y_true,list(map(lambda x: 1 if x > 0.23 else 0, y_hat)), average='macro'), True
    if model_name=='lgb':
        params = {'num_leaves': 256,
              'min_child_samples': 79,
              'objective': 'binary',
              'max_depth': 13,
              'learning_rate': 0.01,
              "boosting_type": "gbdt",
              "subsample_freq": 3,
              "subsample": 0.9,
              "bagging_seed": 11,
              "metric": 'auc',
              "verbosity": -1,
              'reg_alpha': 0.3,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.9,
             }
        for train_idx, val_idx in skf.split(df_train,train_label):
            print('TRAIN:', train_idx, "TEST:", val_idx)
            train = lgb.Dataset(df_train.loc[train_idx, feats], df_train.loc[train_idx, label_name])
            val = lgb.Dataset(df_train.loc[val_idx, feats], df_train.loc[val_idx, label_name])
            model = lgb.train(params, train, valid_sets=val, num_boost_round=5000,
                              early_stopping_rounds=200, verbose_eval=1000
                       #       ,feval=eval
                             )
            oof[val_idx] = model.predict(df_train.loc[val_idx, feats])
            importance += model.feature_importance(importance_type='gain')/n_fold
            pred_y += model.predict(df_test[feats])/n_fold
        return oof,pred_y,importance
    elif model_name=='lgbmc':
        params = {
        'learning_rate': 0.03,
        'metric': 'auc',
        'n_estimators':520,
        'num_leaves':63,
        'bagging_fraction':0.8,
        "feature_fraction":0.8,

        }
        for train_idx, val_idx in skf.split(df_train[feat],train_label):
            print('TRAIN:', train_idx, "TEST:", val_idx)
            train_x, val_x = df_train[feat].iloc[train_idx], df_train[feat].iloc[val_idx]
            train_y, val_y = train_label[train_idx], train_label[val_idx]
            model=lgb.LGBMClassifier(**params)
            model.fit(train_x, train_y,eval_metric='auc', eval_set=[(val_x,val_y)],early_stopping_rounds=100)
            oof[val_idx] = model.predict_proba(val_x)[:,1]
            pred_y += model.predict_proba(df_test[feat])[:,1]/n_fold
        return oof,pred_y
    elif model_name=="cat":

        i=0
        a=pd.DataFrame()
        a['importance']=0
        for train_idx, val_idx in skf.split(df_train,train_label):
            X_train,X_valid=df_train.loc[train_idx, feats],df_train.loc[val_idx, feats]
            y_train,y_valid=df_train.loc[train_idx, label_name],df_train.loc[val_idx, label_name]
                                                                               
            model=CatBoostClassifier(iterations=2500,eval_metric='AUC',
                                     logging_level='Verbose',  
                                        learning_rate=0.05,
                                     depth=6, l2_leaf_reg=5, loss_function='Logloss',
                                     
                                     #task_type='GPU',
                           #devices='0',


                        )
            model.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=100,verbose=500
        )
            #model_dir=f"model/cat/catmodel_{i}.pkl"
            #pkl_save(model_dir,model)
            if i==0:
                a['name'] = model.feature_names_
                a['importance'] = model.feature_importances_
            else:
                a['importance'] += model.feature_importances_
            i=i+1
            oof[val_idx] = model.predict_proba(X_valid)[:,1]
            pred_y += model.predict_proba(df_test[feat])[:,1]/n_fold

            

        a=a.sort_values('importance')
        return oof,pred_y,a
    elif model_name=="xgb":


        for train_idx, val_idx in skf.split(df_train,train_label):
            X_train,X_valid=df_train.loc[train_idx, feats],df_train.loc[val_idx, feats]
            y_train,y_valid=df_train.loc[train_idx, label_name],df_train.loc[val_idx, label_name]
                                                                               
            model=xgb.XGBClassifier(n_estimators=2000, 
                        max_depth=13, 
                        #objective='multi:softprob',
                        seed=0,  
                        nthread=-1, 
                        learning_rate=0.01, 
                        min_child_weight=6,
                        min_child_samples=79,
                        num_leaves = 256, 
                        scale_pos_weight = (23163/200)
                               )
            model.fit(X_train, y_train,eval_set=[(X_valid, y_valid)],early_stopping_rounds=200,verbose=2000,eval_metric="auc")
            oof[val_idx] = model.predict_proba(X_valid)[:,1]
            pred_y += model.predict_proba(df_test[feat])[:,1]/n_fold
        return oof,pred_y    
#总体测试效果
oof,pred_y,importance=model_train(df_train,df_test,train_label,"lgb",feats,seed=5,label_name='LABEL',n_fold=5)
#oof,pred_y,importance=model_train(df_train,df_test,train_label,"cat",feats,seed=2021,n_fold=10)
#oof,pred_y=model_train(df_train,df_test,train_label,"xgb",feats,seed=2021,n_fold=5)

157
TRAIN: [    0     1     3 ... 39996 39997 39998] TEST: [    2     7    14 ... 39992 39994 39999]
Training until validation scores don't improve for 200 rounds


In [ ]:
df_test["LABEL"] = pred_y

In [ ]:
columns = ["CUST_UID","LABEL"]
df_test[columns].to_csv('submit_lgb_seed5_last225_.txt',sep=' ',index=0,header=0)

In [ ]:
#融合

In [22]:
import pandas as pd
lgb1 = pd.read_csv('submit_lgb_seed5_last225_.txt',sep=' ',header=None)
sub = pd.read_csv('lgb_B_3seed.txt',sep=' ',header=None)
lgb1.columns = ["CUST_UID","LABEL"]

In [23]:
sub['LABEL'] = sub['LABEL']*0.8 + lgb1['LABEL']*0.2

In [24]:
sub

,CUST_UID,LABEL
0,f5dcfe9b9e8d496d893cfcba8fcac1d4,0.127902
1,68883d45646d44d8b851476058e4500d,0.019958
2,2b93dfa58f10463a937cd42cc42e54f7,0.143884
3,94fc13e16dac4642ab7544d8889e8579,0.018563
4,6be5af4911964a31a0b5cf115799ace6,0.189754
...,...,...
11995,01d799cc3246416387f2ff9a1468ede1,0.087486
11996,55dba67ea8eb464a810c4fea9331e443,0.091196
11997,527e025840b14373890844a1138197ab,0.151343
11998,89a4e87c9a764c6c9eafda790c76a4e6,0.057707


In [25]:
sub.to_csv('lgb_B.txt', header=None,index=False,encoding = 'utf-8',sep = '\t')